In [1]:
### Imports ###

import pandas as pd
import os
import numpy as np
from ast import literal_eval
import ast

### Changing the work directory ###

"""
Kindly change your working directory here 
"""
os.chdir("D:\\UniSaarland\\incar-dialogue")

In [2]:
### Upload the json files ###

"""
I have read the json files as pandas dataframe for pre-processing 
"""
train_df = pd.read_json('train.json')
valid_df = pd.read_json('val.json')
test_df = pd.read_json('test.json')
train_df = train_df.transpose()
valid_df = valid_df.transpose()
test_df = test_df.transpose()
#train_df.head()


In [3]:
class BuildDataset:
    """
    Contains methods for building the datasets for train, valid and test 
    """
    
    def __init__(self):
        pass
    
    def GenerateLabels(self,df):
        """"
        This method generates labels for every utterance using the reference entities and the knowledge graph 
        
        Arguments:
        df : the dataframe for which labels are to be generated 
        """
        labels = []
        for index, row in df.iterrows():
            labels_kg = []
            if row["reference-entities"] == []:
                 row["labels"] = []
            else:
                for entity in row["reference-entities"]:
                    for triple in row["kg"]:
                        if entity in triple:
                            labels_kg.append(triple)
            labels.append(labels_kg)
        df["labels"] = labels
        return df
    
    
    def PreprocessDataset(self,df):
        
        """"
        This method creates context for utterances in a given conversation 
        
        Arguments:
        df : the dataframe to be pre-processed 
        """
        
        cols = ['task', 'utterance', 'context','reference-entities','kg']
        data = []
        for rowNumber, conversation in enumerate(df.utterances):
            context = []
            for numberofUtterances, utterance in enumerate(conversation):
                if numberofUtterances == 0 :
                    data.append([df.task.iloc[rowNumber],utterance["user"],[],utterance["reference_entities"],df.kg.iloc[rowNumber]])
                    context.append([utterance["user"],utterance["response"]])

                else:
                    data.append([df.task.iloc[rowNumber],utterance["user"],literal_eval(','.join(map(str, context))),utterance["reference_entities"],df.kg.iloc[rowNumber]])
                    context.append([utterance["user"],utterance["response"]])

        df_processed = pd.DataFrame(data,columns=cols)
        #df_processed['context'] = df_processed['context'].apply(ast.literal_eval)
        df_processed['context'] = df_processed['context'].map(np.ravel)
        print(type(df_processed))
        df_final = self.GenerateLabels(df_processed)
        
        return df_final 
    

        


In [4]:
### pre-process the datasets by initializing the BuildDataset class ###
buildDataset = BuildDataset()
df_train = buildDataset.PreprocessDataset(train_df)
df_valid = buildDataset.PreprocessDataset(valid_df)
df_test = buildDataset.PreprocessDataset(test_df)

### save the datasets as .csv files ###
df_train.to_csv("train.csv",index = False)
df_valid.to_csv("valid.csv",index = False)
df_test.to_csv("test.csv",index = False)

print("Datasets created and saved ...")


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
Datasets created and saved ...
